# Calling Kestrel via OpenC2
OpenC2 is the standard, vendor agnostic language for command and control for cyber defense from OASIS. OpenC2 commands are instructions to execute on an system, and are composed of two main components: an `action` and a `target`. The Kestrel Actuator profile supports the `investigate` action. Targets are cyber observables, such as files, decides, IP addresses and processes. In an OpenC2 implementation, there are two main components, a `producer` of a command, and a `consumer`, which executes the command, and returns a `response`. The transport and serialization is an implementation detail. In our example, we use HTTP and JSON. 

Let's get started! First, you'll need to run the consumer

`$ python consumer.py`

Do that in a separate terminal, and return here.

In [ ]:
import sys
import requests
import uuid
import json
import time

import openc2

In [ ]:
host = 'localhost'
port = '5000'

In [ ]:
def get_uuid():
    return str(uuid.uuid4())

def get_headers(uuid):
    headers = {
        "request_id": uuid,
        "created": time.time()
    }
    return headers

### Query Command
Every OpenC2 consumer must suppor the `query` command which specifies what profiles, actions, and targets it supports. 

In [ ]:
def _get_query():
    cmd = {
        "action": "query",
        "target": {"features": ["versions", "profiles", "rate_limit", "pairs"]},
        "args": {},
        "actuator": {
            "x-kestrel": {}
        }
    }
    return cmd

In [ ]:
def run_command(cmd):
    uuid = get_uuid()
    headers = {
        "Content-type": "application/openc2+json;version=1.0",
        "Accept": "application/openc2+json;version=1.0",
        "X-Request-ID": uuid
    }
    body = {
        "headers": get_headers(uuid),
        "body": {"openc2": { "request": cmd } }
        }

    if isinstance(body, str):
        res = requests.post(f'http://{host}:{port}/v1/openc2', data=body, headers=headers)
    else:
        res = requests.post(f'http://{host}:{port}/v1/openc2', json=body, headers=headers)
    print('response from server:',res.status_code)
    return res.json()['body']['openc2']

In [ ]:
profile = run_command(_get_query())
print(profile)

In [ ]:
def _get_investigate():
    cmd = {
        "action": "investigate",
        "target": {
            "ipv4_connection": {
                "src_addr": "172.17.0.2",
                "src_port": 0,
                "dst_addr": "104.244.42.130",
                "dst_port": 0,
                "protocol": "tcp"
                }
        },
        "args": {
            "huntargs": {},
            "returnvars": ["nt_to_investigate"],
            "response_requested": "complete"
            },
        "actuator": {
            "x-kestrel": {
                "huntbook": "exfil"
            }
        }
    }
    return cmd

In [ ]:
response = run_command(_get_investigate())
print(response)

In [ ]:
hb = profile['results']['x-kestrel']['exfil']
hb

In [ ]:
from openc2.v10 import IPv4Connection
from oc2.custom import KestrelArgs, KestrelActuator

In [ ]:
huntbook = 'exfil'
huntargs = {}
returnvars = hb['retvars']
target = IPv4Connection(src_addr='172.17.0.2', dst_addr='104.244.42.130', src_port=0, dst_port=0, protocol='tcp')

cmd = openc2.v10.Command(
    action="investigate",
    target=target,
    args=KestrelArgs(huntargs=huntargs, returnvars=returnvars, response_requested="complete"),
    actuator=KestrelActuator(huntbook=huntbook, 
    )
)

In [ ]:
cmd.serialize()

In [ ]:
response = run_command(cmd.serialize())
print(response)